In [11]:
import os
import numpy as np

import tensorflow as tf
import keras

from keras.applications import VGG16, VGG19, ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imagenet_utils import decode_predictions
from imagenet_utils import preprocess_input

from tqdm import tqdm
import glob

# Data Prep

### Using ImageDataGenerator.

Eventually want to use args and argparse to take in train_dir and test_dir.

In [3]:
train_dir = 'train_CAL/'
test_dir = 'test_CAL/'

In [4]:
IM_HEIGHT = 224
IM_WIDTH = 224
NB_EPOCHS = 1
BAT_SIZE = 16
FC_SIZE = 500 # May need to train this parameter
nb_classes = 1

In [5]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [6]:
nb_train_samples = get_nb_files(train_dir)
nb_classes = len(glob.glob(train_dir + "/*"))
nb_val_samples = get_nb_files(test_dir)

# VGG Model

In [7]:
def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top
    nb_classes: # of classes
    Returns:
    new keras model with last layer
    """
    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x)
    predictions = Dense(nb_classes, activation='sigmoid')(x) 
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [8]:
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    for layer in base_model.layers:
        layer.trainable = False
        model.compile(optimizer='adam',    
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])

In [28]:
# ignore this for now
def setup_to_finetune(model):
    """
    If we want to fine-tine
    """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]: # need to declare num layers to train
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=keras.optimizers.Adam(lr=0.0001, momentum=0.9),   
                 loss='binary_crossentropy')

Need to tune:
* architecture type (ResNet, Inception, VGG19)
* FC_SIZE
* Adam parameters (momentum, learning rate, etc)
* image data generator transforming parameters

In [10]:
def prepare_training(train_dir, test_dir, imheight=IM_HEIGHT, imwidth=IM_WIDTH, batch=BAT_SIZE):
    train_datagen =  ImageDataGenerator(
    #rotation_range=30,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #zoom_range=0.2,
    #rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True)
    
    test_datagen = ImageDataGenerator(
        #rotation_range=30,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #shear_range=0.2,
        #zoom_range=0.2,
        #rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=BAT_SIZE
    )
    validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=BAT_SIZE
    )
    
    return (train_generator, validation_generator)

In [13]:
def vgg16_train(train_dir, val_dir):
    
    train_gen, val_gen = prepare_training(train_dir=train_dir, test_dir=test_dir)

    base_model = VGG16(weights='imagenet', include_top=False)
    #base_model = VGG16(weights=None, include_top=False)
    
    model = add_new_last_layer(base_model, nb_classes)

    setup_to_transfer_learn(model, base_model)

    history = model.fit_generator(train_gen, epochs = NB_EPOCHS, steps_per_epoch = nb_train_samples, \
                                     validation_data=validation_gen, validation_steps = nb_val_samples)
    model.save("vgg19.h5")
    
    return history

h = vgg16_train(train_dir, test_dir)

Found 362 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Epoch 1/1
 12/720 [..............................] - ETA: 4:38:40 - loss: 6.3493 - acc: 0.40 - ETA: 4:15:12 - loss: 7.1603 - acc: 0.45 - ETA: 4:08:23 - loss: 7.4306 - acc: 0.46 - ETA: 3:59:18 - loss: 7.5657 - acc: 0.47 - ETA: 3:53:21 - loss: 7.6468 - acc: 0.48 - ETA: 3:51:09 - loss: 7.7009 - acc: 0.48 - ETA: 3:46:24 - loss: 7.7395 - acc: 0.48 - ETA: 3:45:36 - loss: 7.7685 - acc: 0.48 - ETA: 3:44:26 - loss: 7.7910 - acc: 0.48 - ETA: 3:35:43 - loss: 7.8090 - acc: 0.49 - ETA: 3:36:58 - loss: 7.8238 - acc: 0.49 - ETA: 3:36:37 - loss: 7.8360 - acc: 0.4922

KeyboardInterrupt: 

In [12]:
def vgg19_train(train_dir, val_dir):
    
    train_gen, val_gen = prepare_training(train_dir=train_dir, test_dir=test_dir)

    base_model = VGG19(weights='imagenet', include_top=False)
    model = add_new_last_layer(base_model, nb_classes)

    setup_to_transfer_learn(model, base_model)

    history = model.fit_generator(train_gen, epochs = NB_EPOCHS, steps_per_epoch = nb_train_samples, \
                                     validation_data=validation_gen, validation_steps = nb_val_samples)
    model.save("vgg19.h5")
    
    return history

h = vgg19_train(train_dir, test_dir)

Found 720 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
 3137536/80134624 [>.............................] - ETA: 14:0 - ETA: 9:3 - ETA: 11:5 - ETA: 8:1 - ETA: 25:1 - ETA: 22:3 - ETA: 21:0 - ETA: 21:0 - ETA: 19:0 - ETA: 19:1 - ETA: 18:4 - ETA: 17:4 - ETA: 17:4 - ETA: 18:1 - ETA: 25:2 - ETA: 25:1 - ETA: 23:4 - ETA: 23:0 - ETA: 22:2 - ETA: 22:0 - ETA: 22:1 - ETA: 24:4 - ETA: 24:1 - ETA: 23:2 - ETA: 26:0 - ETA: 26:0 - ETA: 25:5 - ETA: 25:3 - ETA: 25:1 - ETA: 24:5 - ETA: 24:4 - ETA: 24:1 - ETA: 23:5 - ETA: 23:2 - ETA: 23:1 - ETA: 22:4 - ETA: 22:3 - ETA: 22:1 - ETA: 21:5 - ETA: 21:2 - ETA: 21:2 - ETA: 21:0 - ETA: 20:5 - ETA: 20:3 - ETA: 20:2 - ETA: 20:0 - ETA: 19:4 - ETA: 19:4 - ETA: 19:1 - ETA: 19:0 - ETA: 18:5 - ETA: 18:3 - ETA: 18:3 - ETA: 18:2 - ETA: 18:0 - ETA: 17:5 - ETA: 17:3 - ETA: 17:2 - ETA: 17:1 - ETA: 17:0 - ETA: 16:5 - ETA: 16:5 - ETA: 16:4 - ETA: 16:1 - ETA: 16:0 - ETA: 15:5 - ETA: 15:4 - ETA: 15:4 - ETA: 15:2 - ETA: 15:1 - ETA: 14:5 - ETA: 14:3 - ETA

KeyboardInterrupt: 

In [ ]:
def resnet_train(train_dir, val_dir):
    
    train_gen, val_gen = prepare_training(train_dir=train_dir, test_dir=test_dir)

    base_model = ResNet50(weights='imagenet', include_top=False)
    model = add_new_last_layer(base_model, nb_classes)

    setup_to_transfer_learn(model, base_model)

    history = model.fit_generator(train_gen, epochs = NB_EPOCHS, steps_per_epoch = nb_train_samples, \
                                     validation_data=val_gen, validation_steps = nb_val_samples)
    model.save("resnet.h5")
    
    return history

h = resnet_train(train_dir, test_dir)

Found 720 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Epoch 1/1


In [ ]:
def predict(test_dir, model):
    from keras.models import load_model
    test_datagen = ImageDataGenerator(vertical_flip=True)
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=BAT_SIZE,
    )
    
    model = load_model(model) # i.e. vgg16.h5
    pred = model.predict_generator(self, generator, steps, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
    
    return pred

In [70]:
# ignore
#preds = [model.predict(image) for image in tqdm(train_data2)]
#results = [decode_predictions(pred)[0] for pred in preds]
#results